# Searching the Catalog

The `Descartes Labs Catalog API` is a single interface through which you can discover existing raster `Products`, search and retrieve their associated `Images`, and manage your own `Products`.

This guide is meant to serve as an introduction to searching and retrieving raster data. For a more in depth overview of all of `Catalog's` classes and their capabilities please visit the [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html) in our Documentation page.

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Image, Product, properties as p

## Catalog Products
The foundational class within `Descartes Labs Catalog` is a `Product`. Think of a `Product` as a collection of imagery from the same data source, such as [Sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2), but more specifically with the same formatting level, such as  [Sentinel-2 Level 2A](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a).

Below, we will start with the `Product` ID for [Sentinel-2 L2A](https://app.descarteslabs.com/explorer/datasets/esa:sentinel-2:l2a:v1). You can find `Products` through our graphical user interface [Explorer](https://app.descarteslabs.com/explorer/) or through a [`Search`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Poduct.search). 

In [ ]:
s2_product = Product.get("esa:sentinel-2:l2a:v1")
s2_product

A `Product` contains a number of [`Images`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html), which themselves contain the same number of [`Bands`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/band.html). 

Let's see how many `Images` and `Bands` there are in this `Product` through accessing the respective `Product.images()` and `Product.bands()` methods. Note that the type of each is their respective `Search` object:

In [ ]:
s2_images = s2_product.images()
s2_bands = s2_product.bands()

print(f"Number of Images: {s2_images.count()}")
print(f"Type: {type(s2_images)}")
print(f"Number of Bands: {s2_bands.count()}")
print(f"Type: {type(s2_bands)}")

## GeoContexts
Once a `Product` is defined, searching over spatiotemporal extents is simple. The foundational `GeoContext` class is an [`AOI`](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.AOI), which specifies spatial parameters to define an output raster data such as `Resolution`, `CRS`, and a `geometry` cutline.

Here we will create an `AOI` from a `GeoJSON geometry` over the Washington, DC area. Note, we could also pass a `shapely geometry` here if we so chose.

In [ ]:
geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [-77.06237256372073, 38.83807622810744],
            [-76.96982073518272, 38.83807622810744],
            [-76.96982073518272, 38.921715779114066],
            [-77.06237256372073, 38.921715779114066],
            [-77.06237256372073, 38.83807622810744],
        ]
    ],
}

aoi = dl.geo.AOI(geometry, resolution=30.0, crs="EPSG:3857")
aoi

## Finding Images
Now that we have both a `Product` to search and a `GeoContext` over which we want to search and pull imagery, we can chain a series of [`properties` filters](https://docs.descarteslabs.com/descarteslabs/catalog/docs/search.html#descarteslabs.catalog.Search.filter) to return an `ImageCollection`. 

Let's take a look at all the imagery acquired in the month of June, 2023 with less than 10% cloud cover:

In [ ]:
search = (
    s2_images.intersects(aoi)
    .filter("2023-06-01" < p.acquired < "2023-07-01")
    .filter(p.cloud_fraction < 0.1)
)

image_collection = search.collect()
image_collection

There are other attributes useful to filter by, documented in the API reference for [Image](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.Image)

## Rastering Data - Mosaics
Our [`ImageCollection`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageCollection) now has all of the methods we need to retrieve our pixel data. 

First, we'll explore the [`.mosaic()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageCollection.mosaic) method, which will load all specified bands into a single 3D ndarray:

In [ ]:
rgb = image_collection.mosaic(bands=["red", "green", "blue"])
dl.utils.display(rgb, size=5)

The default shape of the resulting ndarray is `(nbands, ny, nx)`:

In [ ]:
rgb.shape

Alternatively, call [`.download_mosaic()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageCollection.download_mosaic) to download this as a GeoTIFF:

In [ ]:
image_collection.download_mosaic(bands=["red", "green", "blue"], dest="data/rgb.tif")

## Rastering Data - Stacks
If we instead wanted to retrieve each `Image` in our `ImageCollection` as a 4D ndarray we use [`ImageCollection.stack`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageCollection.stack) to return a ndarray of shape `(nimages, nbands, ny, nx)`:

In [ ]:
rgb_stack = image_collection.stack(bands=["red", "green", "blue"])
rgb_stack.shape

## ImageCollection Properties
We can iterate over the `properties` of our `ImageCollection` as well:

In [ ]:
dates = list(image_collection.each.acquired.strftime("%Y-%m-%d %H-%m-%s"))
dates

Let's take a look at each `Image` in our `ImageCollection` to explore the data in more detail:

In [ ]:
ids = list(image_collection.each.id)
titles = [f"{ids[i]}\n{dates[i]}" for i in range(len(ids))]

dl.utils.display(*rgb_stack, title=titles, size=5)

Note that each `Image` in the `ImageCollection` may not completely overlap our input `AOI`, as these are individual Sentinel-2 scenes